In [1]:
import gc
import pickle
import time
import json
import cv2
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data.dataloader import DataLoader


In [13]:
# !rm -r sd_xl

In [14]:
!gdown 1W8JYgjDgTWLFCvchHHZ2P_uhS3VS-eGt
!unzip -o sd_xl.zip

Downloading...
From: https://drive.google.com/uc?id=1W8JYgjDgTWLFCvchHHZ2P_uhS3VS-eGt
To: /content/sd_xl.zip
100% 8.98M/8.98M [00:00<00:00, 290MB/s]
Archive:  sd_xl.zip
  inflating: sd_xl/road66_sd.png     
  inflating: sd_xl/road66_sd_1.png   
  inflating: sd_xl/road66_sd_10.png  
  inflating: sd_xl/road66_sd_11.png  
  inflating: sd_xl/road66_sd_12.png  
  inflating: sd_xl/road66_sd_13.png  
  inflating: sd_xl/road66_sd_14.png  
  inflating: sd_xl/road66_sd_15.png  
  inflating: sd_xl/road66_sd_16.png  
  inflating: sd_xl/road66_sd_17.png  
  inflating: sd_xl/road66_sd_18.png  
  inflating: sd_xl/road66_sd_19.png  
  inflating: sd_xl/road66_sd_2.png   
  inflating: sd_xl/road66_sd_3.png   
  inflating: sd_xl/road66_sd_4.png   
  inflating: sd_xl/road66_sd_5.png   
  inflating: sd_xl/road66_sd_6.png   
  inflating: sd_xl/road66_sd_7.png   
  inflating: sd_xl/road66_sd_8.png   
  inflating: sd_xl/road66_sd_9.png   


In [2]:
!gdown 1C0k77EeZrByBUdv36IxS9PiLUvZXRr24
!unzip -o model.zip

!gdown 1Simsmw-H4zKgF5FtJ934bthYulzRlkqz
!unzip -o diffusion_imgs.zip

Downloading...
From: https://drive.google.com/uc?id=1C0k77EeZrByBUdv36IxS9PiLUvZXRr24
To: /content/model.zip
100% 129M/129M [00:00<00:00, 193MB/s]
Archive:  model.zip
  inflating: model_gtsrb.pth         
  inflating: model_lisa.pth          
  inflating: adv_model_gtsrb.pth     
  inflating: adv_model_lisa.pth      
Downloading...
From: https://drive.google.com/uc?id=1Simsmw-H4zKgF5FtJ934bthYulzRlkqz
To: /content/diffusion_imgs.zip
100% 2.88M/2.88M [00:00<00:00, 234MB/s]
Archive:  diffusion_imgs.zip
  inflating: getimg_ai_img-qbD8yQW1ct0CQgu6BgPCH.png  
  inflating: getimg_ai_img-r9pqFJTHiZBSnBJoZBmoz (1).png  
  inflating: getimg_ai_img-SiN26Z5uuPC2iKTYWBmua.png  
  inflating: params.json             
  inflating: stop_sign.jpg           
  inflating: stop_sign2.jpg          
  inflating: stop_sign3.jpg          
  inflating: getimg_ai_img-6WPoZYps7fSXmhggtSkFZ (1).png  
  inflating: getimg_ai_img-deNGXpq62npdoz2P75ChF.png  
  inflating: getimg_ai_img-mJM6Vor9wKKOIbWobiHM4.png  


In [3]:
with open('params.json', 'r') as config:
    params = json.load(config)
    class_n_gtsrb = params['GTSRB']['class_n']
    device = params['device']
    # position_list, _ = load_mask()

class GtsrbCNN(nn.Module):
    def __init__(self, n_class):

        super().__init__()
        self.color_map = nn.Conv2d(3, 3, (1, 1), stride=(1, 1), padding=0)
        self.module1 = nn.Sequential(
            nn.Conv2d(3, 32, (5, 5), stride=(1, 1), padding=2),
            nn.ReLU(),
            nn.Conv2d(32, 32, (5, 5), stride=(1, 1), padding=2),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.ReLU(),
            nn.Dropout(p=0.5),
        )
        self.module2 = nn.Sequential(
            nn.Conv2d(32, 64, (5, 5), stride=(1, 1), padding=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, (5, 5), stride=(1, 1), padding=2),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.ReLU(),
            nn.Dropout(p=0.5),
        )
        self.module3 = nn.Sequential(
            nn.Conv2d(64, 128, (5, 5), stride=(1, 1), padding=2),
            nn.ReLU(),
            nn.Conv2d(128, 128, (5, 5), stride=(1, 1), padding=2),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.ReLU(),
            nn.Dropout(p=0.5),
        )
        self.fc1 = nn.Sequential(
            nn.Linear(14336, 1024, bias=True),
            nn.ReLU(),
            nn.Dropout(p=0.5)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(1024, 1024, bias=True),
            nn.ReLU(),
            nn.Dropout(p=0.5),
        )
        self.fc3 = nn.Linear(1024, n_class, bias=True)

    def forward(self, x):

        x = self.color_map(x)
        branch1 = self.module1(x)
        branch2 = self.module2(branch1)
        branch3 = self.module3(branch2)

        branch1 = branch1.reshape(branch1.shape[0], -1)
        branch2 = branch2.reshape(branch2.shape[0], -1)
        branch3 = branch3.reshape(branch3.shape[0], -1)
        concat = torch.cat([branch1, branch2, branch3], 1)

        out = self.fc1(concat)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

def pre_process_image(image):

    image[:, :, 0] = cv2.equalizeHist(image[:, :, 0])
    image[:, :, 1] = cv2.equalizeHist(image[:, :, 1])
    image[:, :, 2] = cv2.equalizeHist(image[:, :, 2])
    image = image / 255. - .5
    return image

def transform_image(image, ang_range, shear_range, trans_range, preprocess):

    # Rotation
    ang_rot = np.random.uniform(ang_range) - ang_range / 2
    rows, cols, ch = image.shape
    rot_m = cv2.getRotationMatrix2D((cols / 2, rows / 2), ang_rot, 1)

    # Translation
    tr_x = trans_range * np.random.uniform() - trans_range / 2
    tr_y = trans_range * np.random.uniform() - trans_range / 2
    trans_m = np.float32([[1, 0, tr_x], [0, 1, tr_y]])

    # Shear
    pts1 = np.float32([[5, 5], [20, 5], [5, 20]])

    pt1 = 5 + shear_range * np.random.uniform() - shear_range / 2
    pt2 = 20 + shear_range * np.random.uniform() - shear_range / 2

    pts2 = np.float32([[pt1, 5], [pt2, pt1], [5, pt2]])

    shear_m = cv2.getAffineTransform(pts1, pts2)

    image = cv2.warpAffine(image, rot_m, (cols, rows))
    image = cv2.warpAffine(image, trans_m, (cols, rows))
    image = cv2.warpAffine(image, shear_m, (cols, rows))

    image = pre_process_image(image) if preprocess else image

    return image

def test_single_image_gtsrb(img_path, label, adv_model=False):

    trained_model = GtsrbCNN(n_class=class_n_gtsrb).to(device)
    trained_model.load_state_dict(
        torch.load(f'{"adv_" if adv_model else ""}model_gtsrb.pth',
                   map_location=torch.device(device)))
    trained_model.eval()

    img = cv2.imread(img_path)
    img = cv2.resize(img, (32, 32))
    img = pre_process_image(img).astype(np.float32)
    img = transforms.ToTensor()(img)
    img = img.unsqueeze(0).to(device)

    predict = torch.softmax(trained_model(img)[0], 0)
    index = int(torch.argmax(predict).data)
    confidence = float(predict[index].data)

    print(f'Correct: {index==label}', end=' ')
    print(f'Predict: {index} Confidence: {confidence*100}%')

    return index, index == label

In [4]:
with open('params.json', 'r') as config:
    params = json.load(config)
    class_n_lisa = params['LISA']['class_n']
    device = params['device']
    # position_list, _ = load_mask()


class LisaCNN(nn.Module):

    def __init__(self, n_class):

        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, (8, 8), stride=(2, 2), padding=3)
        self.conv2 = nn.Conv2d(64, 128, (6, 6), stride=(2, 2), padding=0)
        self.conv3 = nn.Conv2d(128, 128, (5, 5), stride=(1, 1), padding=0)
        self.fc = nn.Linear(512, n_class)

    def forward(self, x):

        x = nn.ReLU()(self.conv1(x))
        x = nn.ReLU()(self.conv2(x))
        x = nn.ReLU()(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

data_transforms = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize(tuple(mean), tuple(std))
])


def test_single_image_lisa(img_path, ground_truth, adv_model=False):

    trained_model = LisaCNN(n_class=class_n_lisa).to(device)
    trained_model.load_state_dict(
        torch.load(f'{"adv_" if adv_model else ""}model_lisa.pth',
                   map_location=torch.device(device)))
    trained_model.eval()

    img = cv2.imread(img_path)
    img = cv2.resize(img, (32, 32))
    img = transforms.ToTensor()(img)
    img = img.unsqueeze(0).to(device)

    predict = torch.softmax(trained_model(img)[0], 0)
    index = int(torch.argmax(predict).data)
    confidence = float(predict[index].data)

    print(f'Correct: {index==ground_truth}', end=' ')
    print(f'Predict: {index} Confidence: {confidence*100}%')

    return index, index == ground_truth

In [7]:
if __name__ == '__main__':
    directory = './'
    for filename in os.listdir(directory):
        if filename.lower().endswith(('.png', '.jpg')):
            image_path = os.path.join(directory, filename)
            print(image_path)
            test_single_image_gtsrb(image_path, 14, adv_model=False)
            test_single_image_gtsrb(image_path, 14, adv_model=True)
            test_single_image_lisa(image_path, 12, adv_model=False)
            test_single_image_lisa(image_path, 12, adv_model=True)
            print()

./getimg_ai_img-r9pqFJTHiZBSnBJoZBmoz (1).png
Correct: False Predict: 12 Confidence: 86.94319128990173%
Correct: False Predict: 12 Confidence: 92.0737087726593%
Correct: True Predict: 12 Confidence: 23.23242276906967%
Correct: True Predict: 12 Confidence: 40.40265679359436%

./stop_sign3.jpg
Correct: True Predict: 14 Confidence: 28.557923436164856%
Correct: True Predict: 14 Confidence: 93.79168152809143%
Correct: True Predict: 12 Confidence: 24.735748767852783%
Correct: True Predict: 12 Confidence: 27.967163920402527%

./getimg_ai_img-DZNGecKlQjJ2KCffiQymf.png
Correct: True Predict: 14 Confidence: 90.9130871295929%
Correct: False Predict: 13 Confidence: 25.1349538564682%
Correct: False Predict: 10 Confidence: 18.08578222990036%
Correct: True Predict: 12 Confidence: 58.935362100601196%

./getimg_ai_img-uzrz4Pzxm8LipnkHQDHBm.png
Correct: True Predict: 14 Confidence: 90.93763828277588%
Correct: True Predict: 14 Confidence: 95.77475190162659%
Correct: False Predict: 3 Confidence: 17.217044